# Ramsey
## f_LO = §f_LO§ GHz
## T2 = §T2§ µs

In [ ]:
f = h5py.File(§HDF5_PATH§, 'r')
#print(f.keys())

wait  = np.unique(f['data']['wait'][()])*4
freq  = np.unique(f['parameters']['freq'])[()]
data1 = f['data']['I1'][()] + 1j * f['data']['Q1'][()]
data2 = f['data']['I2'][()] + 1j * f['data']['Q2'][()]
data  = data1-data2
#print(freq.shape)
#print(data.shape)

data = IQ_rot(data)
#print(data.shape)

f.close()

def exp_all_freq(x, t2, f0, b, c):
    t_arr = np.array(x[0])
    f_arr = np.array(x[1])
    t2 = np.abs(t2)
    return b*(np.exp(-t_arr/t2)*np.cos(2*np.pi*t_arr*np.abs(f_arr-f0)+c))

x, y  = np.meshgrid(wait, freq)
xdata = np.vstack((x.ravel(), y.ravel()))
ydata = np.real(data).ravel()

popt, pcov = opt.curve_fit(exp_all_freq, xdata, ydata, (13e3, 4.441, 6e-5, 0.)) # TODO: compute these hardcoded values
print('t2, f0, b, c =', popt)

fig, ax = plt.subplots()
wait_fine = np.linspace(wait[0], wait[-1], int(1e3))

def up(fr):
    ax.cla()
    ax.plot(wait, np.real(data[fr]), '.-')
    ax.plot(wait_fine, exp_all_freq((wait_fine,freq[fr]), *popt), 'k--')
    
interactive(up, fr = (0, len(freq)-1, 1))

In [ ]:
result = {'f_LO': 0, 'T2': 0} # TODO: replace these hardcoded values
globals()['result'] = result
result